In [43]:
import numpy as np
import networkx as nx
import string

np_map = np.loadtxt("simple-map-dungeon.txt").astype(int)
np_map

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0,

## Graph creation

In [44]:
G=nx.Graph()

for y in range(np_map.shape[0]):
    for x in range(np_map.shape[1]):
        if np_map[y,x]:
            continue
        
        if y+1 < np_map.shape[0] and np_map[y+1,x] == 0:
            G.add_edge((x,y),(x,y+1))
        
        if y-1 >= 0 and  np_map[y-1,x] == 0:
            G.add_edge((x,y),(x,y-1))
        
        if x+1 < np_map.shape[1] and  np_map[y,x+1] == 0:
            G.add_edge((x,y),(x+1,y))
        
        if x-1 >= 0 and  np_map[y,x-1] == 0:
            G.add_edge((x,y),(x-1,y))
        

In [45]:
paths = nx.shortest_path(G)

## 10.2 - K-medoids clustering

In [52]:
import random

klusters_n = 4
centroids = []

def get_dist_graph(G, v1, v2):
    return len(paths[v1][v2])

for i in range(klusters_n):
    centroids.append(random.choice(list(G.nodes())))

print(centroids)
    
for epoch in range(50):
    clusters = [set() for _ in range(len(centroids))]
    # Loop over the nodes
    for node in G.nodes():
        # Get closest distance to centroid i
        
        min_dist = np.inf
        min_centr = ()
        
        for centr in centroids:
            dist = get_dist_graph(G, node, centr)
            # Save min centroid
            if dist < min_dist:
                min_dist = dist
                min_centr = centr
        
        # Add node to correct cluster
        idx = centroids.index(min_centr)
        clusters[idx].add(node)

    for idx, clust_set in enumerate(clusters):
        min_dist = np.inf
        min_centr = ()
        for node in clust_set:
            sum_dist = sum(get_dist_graph(G, node, node2) for node2 in clust_set)
            if sum_dist < min_dist:
                min_dist = sum_dist
                min_centr = node

        centroids[idx] = min_centr    
print(centroids)        

[(11, 2), (14, 10), (6, 21), (12, 6)]
[(5, 2), (15, 15), (5, 15), (17, 5)]


In [53]:
map_clustered = np_map.copy()
for y in range(np_map.shape[0]):
    for x in range(np_map.shape[1]):
        node = np_map[y,x]
        if node == 0:
            for idx, clust_set in enumerate(clusters):
                if (x,y) in clust_set:
                    map_clustered[y, x] = idx + 1
        else:
            map_clustered[y, x] = 0
map_clustered 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 4, 0, 0, 4, 4, 4, 4, 4, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 0, 0, 4, 4, 4, 4, 4, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 4, 4, 0, 0, 4, 4, 4, 4, 4, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 4, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
       [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 0],
       [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 0],
       [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0],
       [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0],
       [0, 3, 3, 3, 3, 3,

## 10.3 - Spectral clustering

In [56]:
# Adiacent matrix
A = nx.convert_matrix.to_numpy_matrix(G)

# Degree matrix
D = np.zeros_like(A)
# Asssignment of the degrees
for i in range(A.shape[0]):
    D[i,i] = A[i].sum()

# Laplacian matrix
L = D - A

# Eigenvalue decomposition  
e_values, e_vectors = np.linalg.eigh(L)

W = e_vectors[:,1:klusters_n+1]
W

matrix([[-0.11497816,  0.09001546, -0.05210374,  0.02183628],
        [-0.11509498,  0.09058648, -0.05289917,  0.02223714],
        [-0.11466939,  0.08853317, -0.05009633,  0.02083393],
        ...,
        [ 0.03623463,  0.02406908,  0.04147767,  0.05797099],
        [ 0.0361467 ,  0.02368871,  0.03969896,  0.05477532],
        [ 0.03604437,  0.02324901,  0.0376679 ,  0.05114375]])

### K-means clustering over eigenvectors

In [100]:
import random

klusters_n = 4
centroids = []

def get_dist(x1, x2):
    return sum(x1**2 - x2**2)**(1/2)

for i in range(klusters_n):
    idx = np.random.randint(W.shape[0])
    centr = np.squeeze(np.asarray(W[idx]))
    centroids.append(centr)

print(centroids)
    
for epoch in range(50):
    clusters = [set() for _ in range(len(centroids))]
    # Loop over the nodes
    for i in range(W.shape[0]):
        # Get closest distance to centroid i
        
        min_dist = np.inf
        min_centr = ()
        node = np.squeeze(np.asarray(W[i]))

        for centr in centroids:
            dist = get_dist(node, centr)
            # Save min centroid
            if dist < min_dist:
                min_dist = dist
                min_centr = centr
        
        # Add node to correct cluster
        idx = (idx for idx, centr in enumerate(centroids) if centr == min_center)
        clusters[idx].add(node)

    for idx, clust_set in enumerate(clusters):
        sum_nodes = 0
        for node in clust_set:
            sum_nodes += node
        mean = sum_nodes / len(clust_set)
        centroids[idx] = mean    
print(centroids)        

[array([-0.11367679,  0.08381104, -0.04378903,  0.01769841]), array([-0.07924779, -0.06478903,  0.12583468, -0.06227999]), array([0.0352568 , 0.02018674, 0.02626424, 0.03270435]), array([ 0.03232668,  0.0095451 , -0.01010703, -0.02531048])]


TypeError: list indices must be integers or slices, not generator

In [96]:
np.squeeze(np.asarray(W[1]))

array([-0.11509498,  0.09058648, -0.05289917,  0.02223714])

In [95]:
a.flatten()

matrix([[-0.11509498,  0.09058648, -0.05289917,  0.02223714]])